# Sparse Hebbian Learning: plotting tools

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=4, suppress=True)

In [3]:
from shl_scripts.shl_experiments import SHL

DEBUG_DOWNSCALE, verbose = 1, 0

In [4]:
help(SHL)

Help on class SHL in module shl_scripts.shl_experiments:

class SHL(builtins.object)
 |  Base class to define SHL experiments:
 |      - initialization
 |      - coding and learning
 |      - visualization
 |      - quantitative analysis
 |  
 |  Methods defined here:
 |  
 |  __init__(self, height=256, width=256, patch_size=(16, 16), datapath='database/', name_database='kodakdb', n_dictionary=324, learning_algorithm='mp', fit_tol=None, do_precision=True, do_mask=True, l0_sparseness=16, n_iter=16384, eta=0.05, eta_homeo=0.05, nb_quant=128, C=5.0, do_sym=False, alpha_homeo=0.05, max_patches=4096, seed=42, patch_norm=True, batch_size=128, record_each=128, n_image=None, DEBUG_DOWNSCALE=1, verbose=0, data_cache='data_cache', do_HAP=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  code(self, data, dico, coding_algorithm='mp', matname=None, fit_tol=None, l0_sparseness=None)
 |  
 |  decode(self, sparse_code, dico)
 |  
 |  get_data(self, matname=None)
 | 

In [5]:
matname = 'vanilla'
shl = SHL(DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, learning_algorithm='mp', verbose=verbose)
data = shl.get_data(matname=matname)

In [6]:
list_figures = ['show_dico',  'time_plot_prob',  'time_plot_kurt',  'time_plot_var']
dico = shl.learn_dico(matname=matname, list_figures=list_figures)

In [7]:
sparse_code = shl.code(data, dico, matname=matname)
shl.plot_variance(sparse_code);

AttributeError: 'str' object has no attribute 'dictionary'

In [ ]:
shl.plot_variance_histogram(sparse_code);

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts